In [ ]:
from urllib.parse import urlparse
import re
import numpy as np

REAL_URL_JSON_FILE  = "C:\\Users\\ohad\\Downloads\\my_json_real_url"
PHISH_URL_JSON_FILE = "C:\\Users\\ohad\\Downloads\\my_json_phish_url"
FEATURE_FILE        = "C:\\Users\\ohad\\Downloads\\features_file"


def read_json_file(file_name):
    with open(file_name, 'r') as jf:
        lines = jf.read()
    split_lins = lines.split("\n")
    return eval(split_lins[0]),  eval(split_lins[1]),  eval(split_lins[2])


def url_feature_sraper(url, index):
    cur_url_features = []
    parsed_url = urlparse(url)
    url_netloc = parsed_url.netloc.split('.')
    if len(url_netloc) > 2:
        mld = url_netloc[1]
    else:
        mld = url_netloc[0]
    
    protocol = parsed_url.scheme
    if protocol == 'https':
        protocol = 1
    elif protocol == 'http':
        protocol = 2
    else:
        protocol = 3
        
    length_of_mld = len(mld)
    length_of_url = len(url)
    length_of_fqdn = len(parsed_url.netloc)
    count_of_dot_in_url = parsed_url.netloc.count('.') + parsed_url.path.count('.')
    count_of_terms_in_url = len(re.sub("[^0-9a-zA-Z]+", " ", url))
    count_of_terms_in_mld = len(re.sub("[^0-9a-zA-Z]+", " ", mld))
    renking = index
    cur_url_features.append(protocol)
    cur_url_features.append(length_of_mld)
    cur_url_features.append(length_of_url)
    cur_url_features.append(length_of_fqdn)
    cur_url_features.append(count_of_dot_in_url)
    cur_url_features.append(count_of_terms_in_url)
    cur_url_features.append(count_of_terms_in_mld)
    return cur_url_features


def create_feature_lists (final_urls, final_external_urls, final_internal_urls):
    features_url          = []
    external_url_features = []
    internal_url_features = []
    for index, url in enumerate(final_urls):
        features_url.append(url_feature_sraper(url, index))
        curr_external_features = []
        curr_internal_features = []
        for curr_index, ext_url in enumerate(final_external_urls[index]):
            curr_external_features.append(url_feature_sraper(ext_url, curr_index))
        external_url_features.append(curr_external_features)
        for curr_index,int_url in enumerate(final_internal_urls[index]):
            curr_internal_features.append(url_feature_sraper(int_url, curr_index))
        internal_url_features.append(curr_internal_features)
    return features_url, external_url_features, internal_url_features


def extrurct_feathures_from_list(feature_list):
    mean_list   = []
    std_list    = []
    median_list = []
    for curr_feature in feature_list:
        mean_list.append(np.nanmean(curr_feature, axis=0))
        std_list.append(np.std(curr_feature, axis=0))
        median_list.append(np.median(curr_feature, axis=0))
    mean_array   = np.asarray(mean_list)
    std_array    = np.asarray(std_list)
    median_array = np.asarray(median_list)
    return(np.concatenate((mean_array, std_array, median_array), axis = 1))
  
    
def main():
    final_real_urls, final_real_external_urls, final_real_internal_urls = read_json_file(REAL_URL_JSON_FILE)
    final_phish_urls, final_phish_external_urls, final_phish_internal_urls = read_json_file(PHISH_URL_JSON_FILE)

    features_real_url , external_real_url_feature, internal_real_url_feature = create_feature_lists(final_real_urls, final_real_external_urls, final_real_internal_urls)
    features_phish_url, external_phish_url_feature, internal_phish_url_feature = create_feature_lists(final_phish_urls, final_phish_external_urls, final_phish_internal_urls)

    all_origin_url_features   = np.asarray(features_real_url + features_phish_url)
    all_external_url_features = external_real_url_feature + external_phish_url_feature
    all_internal_url_features = internal_real_url_feature + internal_phish_url_feature

    external_calc_fetures = extrurct_feathures_from_list(all_external_url_features)
    internal_calc_fetures = extrurct_feathures_from_list(all_internal_url_features)
    all_calc_fetures = np.concatenate((all_origin_url_features, external_calc_fetures, internal_calc_fetures), axis = 1)
    label_vector = np.zeros(len(features_real_url + features_phish_url))
    label_vector[-len(features_phish_url):] = 1

    np.savez(FEATURE_FILE, all_calc_fetures=all_calc_fetures, label_vector=label_vector)

    
if __name__ == "__main__":
    main()